In [4]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.metrics import *
from collections import Counter

In [5]:
import pandas as pd

data = pd.read_csv('./spam.csv')
print(data)

     Category                                            Message
0         ham  Go until jurong point, crazy.. Available only ...
1         ham                      Ok lar... Joking wif u oni...
2        spam  Free entry in 2 a wkly comp to win FA Cup fina...
3         ham  U dun say so early hor... U c already then say...
4         ham  Nah I don't think he goes to usf, he lives aro...
...       ...                                                ...
5567     spam  This is the 2nd time we have tried 2 contact u...
5568      ham               Will ü b going to esplanade fr home?
5569      ham  Pity, * was in mood for that. So...any other s...
5570      ham  The guy did some bitching but I acted like i'd...
5571      ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [6]:
data['label'] = 0

def transform(text):
    if text == 'spam':
        return 1
    if text == 'ham':
        return 0

In [7]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

stopword = nltk.corpus.stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()

def standardize(text):
    data = []
    for word in text.split():
        if word not in stopword:
            word = lemmatizer.lemmatize(word, "n")
            word = lemmatizer.lemmatize(word, "v")
            word = lemmatizer.lemmatize(word, "a")
            data.append(word)
    return ' '.join(data)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
def preprocess(text):
    text = text.replace({'<.*?>': ''}, regex = True)
    text = text.replace({'[^A-Za-z]': ' '}, regex = True)
    text = text.str.replace('http\S+', ' ')
    text = text.str.replace('\W', ' ')
    text = text.str.replace('\d', ' ')
    text = text.str.lower()
    text = text.apply(standardize)
    return text

data['Message'] = preprocess(data['Message'])
print(data['Message'][0])

go jurong point crazy available bugis n great world la e buffet cine get amore wat


In [9]:
data['label'] = data['Category'].apply(transform)
X = data["Message"]
Y = data["label"]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=0)
print(X_train, X_test, Y_train, Y_test)

1114                          good movie ok leave hourish
3589    free give otherwise nalla adi entey nattil kittum
3095             emigrate something ok maybe bite hopeful
1012                            get home babe still awake
3320                                    kay since already
                              ...                        
4931    hi sexychat girl wait text text great night ch...
3264                                  u gonna get deus ex
1653    ur chance win cash every wk txt action c www m...
2607                    r u sam p eachother meet go house
2732                  mm feel sleepy today shall get dear
Name: Message, Length: 4457, dtype: object 4456    storm msg wen u lift phne u say hello u knw wt...
690           please call immediately urgent message wait
944     also sorta blow couple time recently id rather...
3768                            sir goodmorning free call
1189             come alive good correct good look figure
                             

In [10]:
vocabulary = []
for text in X_train:
    for word in text.split():
        vocabulary.append(word)

dictionary = Counter(vocabulary)
vocabulary = []
for item in dictionary:
    if dictionary[item] >= 50:
        vocabulary.append(item)

print(vocabulary)
print(len(vocabulary))

['good', 'ok', 'leave', 'free', 'give', 'something', 'get', 'home', 'babe', 'still', 'already', 'say', 'oh', 'yeah', 'sorry', 'thing', 'late', 'call', 'even', 'dear', 'look', 'want', 'night', 'pick', 'u', 'da', 'soon', 'co', 'e', 'c', 'way', 'ur', 'ask', 'come', 'x', 'take', 'gonna', 'buy', 'place', 'tonight', 'work', 'time', 'would', 'pls', 'go', 'wait', 'hope', 'great', 'day', 'r', 'tell', 'find', 'much', 'use', 'reply', 'know', 'today', 'right', 'make', 'sure', 'really', 'n', 'help', 'mobile', 'love', 'watch', 'week', 'im', 'thank', 'uk', 'please', 'yes', 'lor', 'keep', 'anything', 'need', 'life', 'think', 'first', 'guy', 'last', 'let', 'nokia', 'win', 'send', 'see', 'min', 'wat', 'miss', 'money', 'service', 'cash', 'prize', 'k', 'stop', 'b', 'morning', 'sleep', 'well', 'number', 'smile', 'like', 'care', 'happy', 'also', 'feel', 'wish', 'hey', 'p', 'msg', 'lol', 'amp', 'tomorrow', 'try', 'message', 'gud', 'show', 'text', 'dont', 'lt', 'gt', 'new', 'year', 'end', 'nice', 'finish', 'o

In [11]:
def filter(text):
    data = []
    for word in text.split():
        if word in vocabulary:
            data.append(word)
    return ' '.join(data)

In [12]:
def filterReview(text):
    text = text.apply(filter)
    return text

X_train = filterReview(X_train)
X_test = filterReview(X_test)
print(X_train)

1114                                        good ok leave
3589                                            free give
3095                                         something ok
1012                                  get home babe still
3320                                              already
                              ...                        
4931    hi wait text text great night send stop stop s...
3264                                          u gonna get
1653                      ur win cash every txt c www x p
2607                                        r u p meet go
2732                                  feel today get dear
Name: Message, Length: 4457, dtype: object


In [13]:
cv = CountVectorizer()
features_CV = cv.fit_transform(X_train)
features_test_cv = cv.transform(X_test)

print(features_CV)

  (0, 35)	1
  (0, 77)	1
  (0, 51)	1
  (1, 29)	1
  (1, 32)	1
  (2, 77)	1
  (2, 95)	1
  (3, 31)	1
  (3, 44)	1
  (3, 5)	1
  (3, 99)	1
  (4, 0)	1
  (5, 31)	1
  (6, 88)	2
  (7, 76)	1
  (7, 131)	1
  (8, 31)	1
  (8, 97)	1
  (8, 106)	1
  (8, 50)	1
  (9, 9)	1
  (9, 23)	1
  (10, 20)	1
  (11, 31)	1
  (11, 56)	1
  :	:
  (4450, 71)	1
  (4450, 132)	1
  (4451, 77)	1
  (4452, 73)	1
  (4452, 119)	1
  (4452, 36)	1
  (4452, 90)	1
  (4452, 91)	1
  (4452, 100)	2
  (4452, 104)	2
  (4452, 43)	1
  (4453, 31)	1
  (4453, 34)	1
  (4454, 116)	1
  (4454, 126)	1
  (4454, 12)	1
  (4454, 114)	1
  (4454, 24)	1
  (4454, 130)	1
  (4455, 33)	1
  (4455, 61)	1
  (4456, 31)	1
  (4456, 20)	1
  (4456, 109)	1
  (4456, 25)	1


In [15]:
model_svm = svm.SVC( C = 100, gamma = 0.01)
model_svm.fit(features_CV, Y_train)

SVC(C=100, gamma=0.01)

In [16]:
Y_preds_svm = model_svm.predict(features_test_cv)
print('SVM:')
print('Accuracy: ', accuracy_score(Y_test, Y_preds_svm)*100)
print('Precision: ', precision_score(Y_test, Y_preds_svm)*100)
print('Recall: ', recall_score(Y_test, Y_preds_svm)*100)
print('Confusion Matrix: ', confusion_matrix(Y_test, Y_preds_svm))
print('********************************************************')

SVM:
Accuracy:  97.75784753363229
Precision:  97.2027972027972
Recall:  86.875
Confusion Matrix:  [[951   4]
 [ 21 139]]
********************************************************


In [18]:
from joblib import dump, load

In [20]:
dump(model_svm, 'model_svm')

['model_svm']